## Solutions - Problem 1

Get number of flights departed from each of the US airport in the month of 2008 January.

* We have to use airport codes to determine US airport.
* We need to use airlines data to get departure details.
* To solve this problem we have to perform inner join.

Let us start spark context for this Notebook so that we can execute the code provided. You can sign up for our [10 node state of the art cluster/labs](https://labs.itversity.com/plans) to learn Spark SQL using our unique integrated LMS.

In [ ]:
from pyspark.sql import SparkSession

import getpass
username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port', '0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    appName(f'{username} | Python - Joining Data Sets'). \
    master('yarn'). \
    getOrCreate()

If you are going to use CLIs, you can use Spark SQL using one of the 3 approaches.

**Using Spark SQL**

```
spark2-sql \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Scala**

```
spark2-shell \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

**Using Pyspark**

```
pyspark2 \
    --master yarn \
    --conf spark.ui.port=0 \
    --conf spark.sql.warehouse.dir=/user/${USER}/warehouse
```

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "2")

In [ ]:
airlinesPath = "/public/airlines_all/airlines-part/flightmonth=200801"

In [ ]:
airlines = spark. \
    read. \
    parquet(airlinesPath)

In [ ]:
airlines. \
    select(
        "Year", "Month", "DayOfMonth", 
        "Origin", "Dest", "CRSDepTime"
    ). \
show()

In [ ]:
airlines.count()

In [ ]:
airportCodesPath = "/public/airlines_all/airport-codes"

In [ ]:
def getValidAirportCodes(airportCodesPath):
    airportCodes = spark. \
        read. \
        option("sep", "\t"). \
        option("header", True). \
        option("inferSchema", True). \
        csv(airportCodesPath). \
        filter("!(State = 'Hawaii' AND IATA = 'Big') AND Country = 'USA'")
    return airportCodes

In [ ]:
airportCodes = getValidAirportCodes(airportCodesPath)

In [ ]:
airportCodes.count()

In [ ]:
from pyspark.sql.functions import col, lit, count

In [ ]:
airlines. \
    join(airportCodes, airportCodes["IATA"] == airlines["Origin"]). \
    select(col("Year"), col("Month"), col("DayOfMonth"), airportCodes["*"], col("CRSDepTime")). \
    show()

In [ ]:
airlines. \
    join(airportCodes, airportCodes["IATA"] == airlines["Origin"]). \
    groupBy("Origin"). \
    agg(count(lit(1)).alias("FlightCount")). \
    orderBy(col("FlightCount").desc()). \
    show()